# 连接度，道路交叉口密度分析

In [ ]:
def analyze_connectivity(G, place_name="研究区域"):
    """
    分析路网连接度和交叉口密度
    """
    # 获取节点和边
    nodes, edges = ox.graph_to_gdfs(G)
    
    # 计算每个节点的度数（连接的道路数量）
    node_degrees = dict(G.degree())
    
    # 将度数信息添加到节点GeoDataFrame
    nodes['degree'] = nodes.index.map(node_degrees)
    
    # 识别交叉口（度数 >= 3的节点）
    intersections = nodes[nodes['degree'] >= 3]
    
    # 创建网格计算交叉口密度
    bounds = nodes.total_bounds
    grid_size = 200  # 米
    x_coords = np.arange(bounds[0], bounds[2], grid_size/111320)
    y_coords = np.arange(bounds[1], bounds[3], grid_size/111320)
    
    intersection_densities = []
    grid_polygons = []
    
    for i in range(len(x_coords)-1):
        for j in range(len(y_coords)-1):
            poly = Polygon([
                (x_coords[i], y_coords[j]),
                (x_coords[i+1], y_coords[j]),
                (x_coords[i+1], y_coords[j+1]),
                (x_coords[i], y_coords[j+1])
            ])
            
            # 计算网格内的交叉口数量
            int_in_grid = intersections[intersections.within(poly)]
            intersection_density = len(int_in_grid) / (grid_size**2 / 1000000)  # 个/平方公里
            
            intersection_densities.append(intersection_density)
            grid_polygons.append(poly)
    
    # 可视化
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 8))
    
    # 交叉口密度热力图
    grid_gdf = gpd.GeoDataFrame({'density': intersection_densities}, 
                               geometry=grid_polygons, crs=nodes.crs)
    
    grid_gdf.plot(column='density', cmap='YlOrRd', ax=ax1, legend=True,
                 legend_kwds={'label': '交叉口密度 (个/km²)', 'shrink': 0.8})
    edges.plot(ax=ax1, color='gray', linewidth=0.3, alpha=0.5)
    ax1.set_title('交叉口密度分布', fontsize=14, fontweight='bold')
    ax1.set_axis_off()
    
    # 节点度数分布
    degree_counts = pd.Series(list(node_degrees.values())).value_counts().sort_index()
    ax2.bar(degree_counts.index, degree_counts.values, color='#2ca25f', alpha=0.7)
    ax2.set_xlabel('节点度数')
    ax2.set_ylabel('节点数量')
    ax2.set_title('节点度数分布', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # 计算连接度指标
    n = len(nodes)  # 节点数
    m = len(edges)  # 边数
    
    # Beta指数（平均连接度）
    beta_index = m / n if n > 0 else 0
    
    # Gamma指数（实际边数与最大可能边数之比）
    max_possible_edges = n * (n - 1) / 2 if n > 1 else 0
    gamma_index = m / max_possible_edges if max_possible_edges > 0 else 0
    
    # 交叉口比例
    intersection_ratio = len(intersections) / n if n > 0 else 0
    
    # 指标可视化
    metrics = ['Beta指数', 'Gamma指数', '交叉口比例']
    values = [beta_index, gamma_index, intersection_ratio]
    
    bars = ax3.bar(metrics, values, color=['#fc8d59', '#ffffbf', '#99d594'])
    ax3.set_ylabel('指标值')
    ax3.set_title('路网连接度指标', fontsize=14, fontweight='bold')
    
    # 在柱子上添加数值标签
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{value:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    print(f"路网连接度分析结果:")
    print(f"- 节点总数: {n}")
    print(f"- 边总数: {m}")
    print(f"- 交叉口数量: {len(intersections)}")
    print(f"- Beta指数: {beta_index:.3f}")
    print(f"- Gamma指数: {gamma_index:.3f}")
    print(f"- 交叉口比例: {intersection_ratio:.3f}")
    
    return intersections, grid_gdf

# 使用示例
# intersections, connectivity_grid = analyze_connectivity(G)